In [1]:
import numpy as np
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import json
import os
import time
import getpass
# Khai báo trình duyệt Chrome
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [ ]:
class FacebookGroupScraper:
    def __init__(self):
        # Khởi tạo lớp FacebookGroupScraper và in thông báo chào mừng
        print("\n === FACEBOOK GROUP MEMBER SCRAPER ===")
        
        # Gọi hàm lấy thông tin cấu hình (như thông tin đăng nhập, ID group)
        self.get_config()  
        
        # Gọi hàm thiết lập trình duyệt (WebDriver)
        self.setup_driver()  

    def get_config(self):
        try:
            # Nhập thông tin đăng nhập (email và mật khẩu)
            print("Nhap thong tin dang nhap")
            
            # Nhập email người dùng và loại bỏ khoảng trắng thừa
            self.email = input("Emil/UserName: ").strip()  
            
            # Nhập mật khẩu người dùng, mật khẩu sẽ không hiển thị trên màn hình
            self.password = getpass.getpass("Password: ")  

            # Nhập ID Group Facebook, đây là nhóm cần thu thập thông tin
            print("\n Nhap ID group facebook:")
            self.group_id = input("Group ID: ").strip()  

            # Nhập số lần scroll để tải thêm thành viên
            print("\nSo lan scroll de load")
            # Nếu người dùng không nhập gì, mặc định số lần cuộn sẽ là 5
            self.scroll_count = int(input("So lan scroll (mac dinh 5)") or "5")  
        except Exception as e:
            # In lỗi nếu có vấn đề trong quá trình nhập thông tin
            print(f"Loi cau hinh {e}")  
            pass
    
    def setup_driver(self):
        try:
            # Khởi tạo WebDriver (trình duyệt Chrome)
            # Đảm bảo rằng bạn đã cài đặt ChromeDriver và thư viện Selenium
            self.driver = webdriver.Chrome()  # Tạo đối tượng WebDriver sử dụng Chrome
            
            # Phóng to cửa sổ trình duyệt
            self.driver.maximize_window()  
        except Exception as e:
            # In lỗi nếu không thể khởi tạo WebDriver
            print(f"Loi khoi tao trinh duyet: {e}")  
    
    def login(self):
        try:
            # Truy cập vào trang đăng nhập của Facebook
            self.driver.get("https://www.facebook.com/") 

            # Nhập email vào ô email
            email_input = self.driver.find_element(By.ID, "email")  # Tìm ô nhập email bằng ID
            email_input.send_keys(self.email)  # Gửi thông tin email vào ô

            # Nhập mật khẩu vào ô password
            pass_input = self.driver.find_element(By.ID, "pass")  # Tìm ô nhập mật khẩu bằng ID
            pass_input.send_keys(self.password)  # Gửi thông tin mật khẩu vào ô

            # Nhấn nút đăng nhập
            login_button = self.driver.find_element(By.NAME, "login")  # Tìm nút đăng nhập
            login_button.click()  # Click vào nút đăng nhập

            # Chờ trong 20 giây để đảm bảo trang tải hoàn tất
            time.sleep(20)  
            print('Dang nhap thanh cong')  # Thông báo đăng nhập thành công
            return True  # Trả về True khi đăng nhập thành công
        except Exception as e:
            # In lỗi nếu có vấn đề khi đăng nhập
            print(f"Loi dang nhap: {e}")  
            return False  # Trả về False nếu đăng nhập thất bại

    def get_group_member(self):
        try:
            # Truy cập trang thành viên của group Facebook theo ID group
            self.driver.get(f"https://www.facebook.com/groups/{self.group_id}/members")  
            
            # Chờ 5 giây để trang tải xong
            time.sleep(5)  

            # Sử dụng set để lưu danh sách thành viên, tránh trùng lặp
            members = set()  

            # Lặp qua số lần scroll được nhập
            for i in range(self.scroll_count):
                # Cuộn trang xuống để tải thêm thành viên
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  
                
                # Chờ 3 giây mỗi lần cuộn trang để tránh bị giới hạn của Facebook
                time.sleep(3)  
                print(f"Scroll lần {i+1}/{self.scroll_count}")  # In thông báo số lần cuộn

                # Thu thập thông tin thành viên sau mỗi lần cuộn trang
                user_elements = self.driver.find_elements(By.CSS_SELECTOR, "a[href*='/user/']")  # Tìm các phần tử liên kết đến trang người dùng
                print(len(user_elements))  # In số lượng thành viên tìm được

                # Lặp qua từng thành viên và thu thập thông tin
                for user in user_elements:
                    try:
                        href = user.get_attribute('href')  # Lấy URL của người dùng
                        print(href)
                        # Kiểm tra nếu URL chứa '/user/', đó là một thành viên
                        if '/user/' in href:  
                            # Tách ID người dùng từ URL
                            user_id = href.split('/user/')[1].strip('/')  
                            print(user_id)  # In ID người dùng
                            
                            # Lấy tên người dùng từ văn bản của phần tử
                            name = user.text  
                            print(name)  # In tên người dùng
                            
                            # Thêm thông tin thành viên vào set
                            members.add((user_id, name))  
                            print(user_id, "-", name)  # In thông tin ID và tên người dùng
                    except Exception:
                        print(f"Không ra thành viên")  # In thông báo lỗi nếu không lấy được thông tin thành viên

            # Xuất danh sách thành viên ra file Excel
            df = pd.DataFrame(members)  # Chuyển danh sách thành viên thành DataFrame
            # Lưu DataFrame vào file Excel
            df.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/Data Crawling/Facebook_Group_Members.xlsx", index=False)  
            print("Lưu thành công")  # Thông báo lưu thành công
            return members  # Trả về danh sách thành viên

        except Exception as e:
            # In lỗi nếu có vấn đề trong quá trình thu thập thông tin thành viên
            print(f"Loi thu thap thanh vien: {e}")  
            return None  # Trả về None nếu không thu thập được thành viên

def main():
    scraper = None  # Khởi tạo biến scraper để lưu đối tượng của FacebookGroupScraper

    try:
        # Tạo đối tượng FacebookGroupScraper
        scraper = FacebookGroupScraper()  
        
        # Nếu đăng nhập thành công
        if scraper.login():  
            # Lấy thông tin thành viên của group
            scraper.get_group_member()  
            print("____________________")  # In dấu phân cách
            time.sleep(10)  # Chờ 10 giây sau khi lấy xong thông tin
        time.sleep(100)  # Chờ 100 giây để tránh thoát chương trình quá nhanh
    except Exception:
        # Bắt lỗi và không làm gì nếu có lỗi xảy ra
        pass  


In [3]:
def main():
    scraper = None
    
    try:
        scraper = FacebookGroupScraper()
        if scraper.login():
            scraper.get_group_member()
            print("____________________")
            time.sleep(10)
        time.sleep(100)
    except Exception:
        pass


In [4]:
if __name__ == "__main__":
    main()


 === FACEBOOK GROUP MEMBER SCRAPER ===
Nhap thong tin dang nhap

 Nhap ID group facebook:

So lan scroll de load
Dang nhap thanh cong
Scroll lần 1/5
51
https://www.facebook.com/groups/1282682436200491/user/100088045819292/
100088045819292

100088045819292 - 
https://www.facebook.com/groups/1282682436200491/user/100085018123405/
100085018123405

100085018123405 - 
https://www.facebook.com/groups/1282682436200491/user/100069263424156/
100069263424156

100069263424156 - 
https://www.facebook.com/groups/1282682436200491/user/100049309627131/
100049309627131

100049309627131 - 
https://www.facebook.com/groups/1282682436200491/user/100047823323262/
100047823323262

100047823323262 - 
https://www.facebook.com/groups/1282682436200491/user/100039158704981/
100039158704981

100039158704981 - 
https://www.facebook.com/groups/1282682436200491/user/100035046482443/
100035046482443

100035046482443 - 
https://www.facebook.com/groups/1282682436200491/user/100031987725528/
100031987725528

1000319877